In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from numpy import genfromtxt   
import nibabel as nb
from brainspace.utils.parcellation import reduce_by_labels
from brainspace.utils.parcellation import map_to_labels
import matplotlib.pyplot as plt
from nilearn import plotting


### get HCP - S900 subject list 

In [2]:
subjlist = '../data/subjectListS900_QC_gr.txt'
f = open(subjlist); mylist = f.read().split("\n"); f.close() 
subjlist = joinedlist = mylist[:-1]

mysubjects = []

for subj in subjlist:
    mysubjects.append(subj)
print(len(mysubjects))


709


### get HCP demographics

In [3]:
HCP_u = pd.read_csv('../data/HCP_solar/HCP_s1200_unrestricted.csv', 
                    index_col = 0)

HCP_u.index = HCP_u.index.map(str)

HCP_r = pd.read_csv('../data/HCP_solar/HCP_s1200_restricted.csv', 
                    index_col = 0)

HCP_r.index = HCP_r.index.map(str)

len(HCP_u.index), len(HCP_r.index)

(1206, 1206)

### get myelin in vivo-measures for subfields and cortex

In [4]:
mye_cortex_fname = '../data/tout_group/H709_mye_cortex.h5'
mye_subfie_fname = '../data/tout_group/H709_mye_subfields.h5'

h1 = h5py.File(mye_cortex_fname, 'r');
h2 = h5py.File(mye_subfie_fname, 'r');

mye_cortex = np.array(h1['data'])
mye_subfie = np.array(h2['data'])

mye_LSUB = mye_subfie[0:1024,:]
mye_LCA  = mye_subfie[1024:1024+2048,:]
mye_LDG  = mye_subfie[1024+2048:1024+2048+1024,:]

mye_LSUB_cortex_concat = np.concatenate((mye_cortex.T, mye_LSUB.T), axis=1)
mye_LCA_cortex_concat  = np.concatenate((mye_cortex.T, mye_LCA.T), axis=1)
mye_LDG_cortex_concat  = np.concatenate((mye_cortex.T, mye_LDG.T), axis=1)

In [5]:
print(mye_LSUB_cortex_concat.shape)
print(mye_LCA_cortex_concat.shape)
print(mye_LDG_cortex_concat.shape)

(709, 1384)
(709, 2408)
(709, 1384)


## prepare cortex - LSUB data for solar analysis

In [6]:
conndir  = '../data/tout_hippoc/' 
odir = '../solar/scov_lsub'

# we'll have 360 nodes (Glasser vertex number)
for i in range(1,361):

    node_str = []
    node_str.append('nodeC_'+ str(i))

    tot_node_num_hipp = 1024
    for j in range(1, tot_node_num_hipp+1):
        node_str.append('nodeH_'+ str(j))
    
    mycols = ['age', 'sex'] + node_str 

    df = pd.DataFrame(index = mysubjects,
                     columns = mycols)
    df.index.name = 'id'
    
    # generate sub dir and fname    
    sub_name  = os.path.join(os.path.join(odir, 'scov_lsub_%s' % str(i)))
    sub_fname = os.path.join(sub_name, 'scov.csv')
    
    if not os.path.isdir(sub_name):
        os.makedirs(sub_name)

    k = 0        

    for subj in subjlist:

        iA = HCP_r.index.get_loc(subj)
        iB = HCP_u.index.get_loc(subj)
        iC = df.index.get_loc(subj)

        df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
        df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']

        mye = np.concatenate((mye_LSUB_cortex_concat[:, (i-1)].reshape(-1,1), 
                              mye_LSUB_cortex_concat[:, 360:]), axis=1)    

        mye = mye[k,:] + 1000
 
        df.iloc[iC][node_str] = mye.reshape(len(mye))     

        k += 1

    # drop NaN's and save
    df = df.dropna()
    df.to_csv(sub_fname)
    

# cortex-LCA

In [7]:
conndir  = '../data/tout_hippoc/' 
odir = '../solar/scov_lca'

# we'll have 360 nodes (Glasser vertex number)
for i in range(1,361):

    node_str = []

    node_str.append('nodeC_'+ str(i))

    tot_node_num_hipp = 2048
    for j in range(1, tot_node_num_hipp+1):
        node_str.append('nodeH_'+ str(j))
    
    mycols = ['age', 'sex'] + node_str 

    df = pd.DataFrame(index = mysubjects,
                     columns = mycols)
    df.index.name = 'id'
    
    # generate sub dir and fname    
    sub_name  = os.path.join(os.path.join(odir, 'scov_lca_%s' % str(i)))
    sub_fname = os.path.join(sub_name, 'scov.csv')
    
    if not os.path.isdir(sub_name):
        os.makedirs(sub_name)

    k = 0        

    for subj in subjlist:

        iA = HCP_r.index.get_loc(subj)
        iB = HCP_u.index.get_loc(subj)
        iC = df.index.get_loc(subj)

        df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
        df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']

        mye = np.concatenate((mye_LCA_cortex_concat[:, (i-1)].reshape(-1,1), 
                              mye_LCA_cortex_concat[:, 360:]), axis=1)    

        mye = mye[k,:] + 1000
 
        df.iloc[iC][node_str] = mye.reshape(len(mye))     

        k += 1

    # drop NaN's and save
    df = df.dropna()
    df.to_csv(sub_fname)
    

# cortex - LDG

In [8]:
conndir  = '../data/tout_hippoc/' 
odir = '../solar/scov_ldg'

for i in range(1,361):

    node_str = []
    node_str.append('nodeC_'+ str(i))

    tot_node_num_hipp = 1024
    for j in range(1, tot_node_num_hipp+1):
        node_str.append('nodeH_'+ str(j))
    
    mycols = ['age', 'sex'] + node_str 

    df = pd.DataFrame(index = mysubjects,
                     columns = mycols)
    df.index.name = 'id'
    
    # generate sub dir and fname    
    sub_name  = os.path.join(os.path.join(odir, 'scov_ldg_%s' % str(i)))
    sub_fname = os.path.join(sub_name, 'scov.csv')
    
    if not os.path.isdir(sub_name):
        os.makedirs(sub_name)

    k = 0        

    for subj in subjlist:

        iA = HCP_r.index.get_loc(subj)
        iB = HCP_u.index.get_loc(subj)
        iC = df.index.get_loc(subj)

        df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
        df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']

        mye = np.concatenate((mye_LDG_cortex_concat[:, (i-1)].reshape(-1,1), 
                              mye_LDG_cortex_concat[:, 360:]), axis=1)    
        mye = mye[k,:] + 1000

        df.iloc[iC][node_str] = mye.reshape(len(mye))     

        k += 1

    # drop NaN's and save
    df = df.dropna()
    df.to_csv(sub_fname)
    